### Check epics connection

In [1]:
from epics import caget

In [2]:
# Make sure these lines are included in your bash_rc or bash_profile.
#######
# alias epicsproxy="ssh -fN -L <some number over 24000>:lcls-prod01.slac.stanford.edu:5068 slac"
# export EPICS_CA_NAME_SERVERS=localhost:<same number over 24000>
# alias epicstest="caget KLYS:LI22:11:KPHR"
#######

## In a terminal, run "epicsproxy" in and enter SLAC credential to establish connection.
## Then run "epicstest". If the connection worked, a number should be returned. 
## If a number is returned, yet this cell fails, re-start jupyter. 

#caget("QUAD:HTR:440:BACT")
caget("OTRS:DIAG0:525:XRMS")

cannot connect to OTRS:DIAG0:525:XRMS


## Choose and slice Bmad lattice

In [1]:
from bmad_live import get_DM_GOOD, get_pvdata, tao_commands, set_htr_twiss, run1

In [2]:
MODEL = 'sc_bsyd' # or sc_diag0 or sc_inj

BEGELE = 'BEGINNING'

#ENDELE = 'OTR0H04'
ENDELE = "ENDCOL0"

## Do not include sections beyond the beam's reach!!!
#ENDELE = 'END'
#ENDELE = 'ENDL3B'

In [3]:
from pytao import Tao

tao = Tao(f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init  -slice  {BEGELE}:{ENDELE}')

# Master switches for element scaling
tao.cmd('set ele quad::* field_master = T')
#tao.cmd('set ele lcavity::CAVL* autoscale_amplitude = T') # field_au
if MODEL == 'sc_inj':
    tao.cmd('set ele lcavity::CAVL* autoscale_phase = T')

In [4]:
MONITOR, DM_GOOD = get_DM_GOOD(tao, MODEL)

Bad PV: QUAD:COL0:280:BACT
Bad PV: ACCL:L0B:0170:AACTMEAN
Bad PV: BPMS:COL0:940:X
Bad PV: QUAD:COL0:560:BACT
Bad PV: BPMS:GUNB:925:X
Bad PV: BPMS:COL0:320:TMIT
Bad PV: QUAD:COL0:800:BACT
Bad PV: BPMS:COL0:800:Y
Bad PV: ACCL:GUNB:455:PACT_AVG
Bad PV: BPMS:HTR:365:X
Bad PV: BPMS:HTR:365:Y
Bad PV: BPMS:COL0:720:Y
Bad PV: BPMS:COL0:400:TMIT
Bad PV: ACCL:L0B:0130:PACTMEAN
Bad PV: QUAD:HTR:980:BACT
Bad PV: QUAD:COL0:940:BACT
Bad PV: BPMS:COL0:880:X
Bad PV: QUAD:HTR:385:BACT
Bad PV: BPMS:COL0:480:Y
Bad PV: BPMS:COL0:720:X
Bad PV: BPMS:HTR:830:Y
Bad PV: BPMS:COL0:260:TMIT
Bad PV: BPMS:GUNB:314:Y
Bad PV: QUAD:L0B:0185:BACT
Bad PV: BPMS:COL0:880:Y
Bad PV: BPMS:HTR:120:X
Bad PV: BPMS:COL0:320:Y
Bad PV: ACCL:L0B:0120:PACTMEAN
Bad PV: BPMS:COL0:280:Y
Bad PV: QUAD:HTR:860:BACT
Bad PV: ACCL:L0B:0110:PACTMEAN
Bad PV: BPMS:HTR:320:TMIT
Bad PV: ACCL:L0B:0140:AACTMEAN
Bad PV: QUAD:HTR:300:BACT
Bad PV: QUAD:COL0:880:BACT
Bad PV: QUAD:GUNB:823:2:BACT
Bad PV: BPMS:COL0:260:X
Bad PV: QUAD:HTR:120:BACT
Bad PV

In [11]:
# DM_GOOD['correctors'].data

In [9]:
# Scale Tao plots using Tao magic

#%%tao
#place top energy
#place middle orbit
#x_s top 0 100
#sc *
#tao.cmd('set global plot_on = F')

In [ ]:
# Updating bmad live-model once

run1(tao, MONITOR, DM_GOOD)

In [10]:
# Match HTR optics to design
set_htr_twiss(tao)

### To obtain the transfer matrix Mx and My for PyEmittance measurement

In [11]:
def get_mats():
    mat6 = tao.matrix('Q0H01#2', 'OTR0H04')['mat6']  # For HTR line
    
    #mat6 = tao.matrix('QDG004#2', 'OTRDG02')['mat6'] # For DIAG0 line
    
    mat2x = mat6[0:2, 0:2]
    mat2y = mat6[2:4, 2:4]
    return mat2x, mat2y

In [12]:
get_mats()

(array([[-3.12456865,  4.01328803],
        [ 1.69661397, -2.49922515]]),
 array([[3.51169476, 5.37844322],
        [1.3110681 , 2.29276912]]))

# Optics compensation

### The example here is for the heater section
### Make sure the real machine has reasonable transmission along this section
##### Quad to be distorted: Q0H01
##### Monitor: OTR0H04
##### Quads to fix optics downstream: 'Q0H05','Q0H06','Q0H07','Q0H08','QHD01'
##### Downstream checkpoint:  ENDHTR


In [72]:
##  Chris's trick to vary the beginning Twiss
##  so that the twiss at OTR0H04 are some reasonable (guess) values

def set_OTR0H04():
    cmds = f"""
set dat OTR0H04.midtwiss[1]|meas = 5
set dat OTR0H04.midtwiss[2]|meas = 0
set dat OTR0H04.midtwiss[3]|meas = 5
set dat OTR0H04.midtwiss[4]|meas = 0
vv
vd
use dat OTR0H04.midtwiss[1:4]
use var begtwiss[1:4]
run 
run
""".split('\n')
    for cmd in cmds:
        tao.cmd(cmd)
    
# set_OTR0H04()

In [12]:
def optimize_twiss_cmds(dat_name, var_name, desired_twiss):
    """
        dat_name: STRING
        var_name: STRING
        desired_twiss : dictionary from tao.ele_twiss("ele_name")
    """
    
    out = f""" 
set global lattice_calc_on = F
!set global optimizer = lmdif 
set global optimizer = lm

veto dat *
veto var *

set dat {dat_name}[1]|meas = {desired_twiss['beta_a']}
set dat {dat_name}[2]|meas = {desired_twiss['alpha_a']}
set dat {dat_name}[3]|meas = {desired_twiss['beta_b']}
set dat {dat_name}[4]|meas = {desired_twiss['alpha_b']}
!!set dat {dat_name}[5]|meas = {desired_twiss['eta_x']}
!!set dat {dat_name}[6]|meas = {desired_twiss['etap_x']}

use dat {dat_name}[1:4]
use var {var_name}

set global lattice_calc_on = T

!show top

run
run
run
run
run
run

!show dat {dat_name}
!show merit
"""
    return out.split('\n')

In [13]:
def twiss_optimizer2(dat_name, var_name, desired_twiss, reset_lat=False, debug=False):
    
    TWISS_CMDS = optimize_twiss_cmds(dat_name, var_name, desired_twiss)
    
    merit = float(tao.merit()[0])
    print(f"Old merit: {merit}")   
    
    tao.cmds(TWISS_CMDS);
    
    merit = float(tao.merit()[0]) 
    assert merit < 1E-16, f"merit did NOT drop low enough: {merit}!!"
    print(f"New merit: {merit}")
    
    var_info = tao.var_v_array(var_name)
    N_var = len(var_info)
    q_names = [var_info[i]['var_attrib_name'][0:5] for i in range(N_var)]
 

    dict_b1_grad = {q_names[i]:tao.ele_gen_attribs(q_names[i])['B1_GRADIENT'] for i in range(N_var)}

    return dict_b1_grad
    

In [90]:
tao.cmd('SLD')               # Set the lattice back to design
run1(tao, MONITOR, DM_GOOD)  # Load in live settings
set_OTR0H04()                # Guess the initial optics ( this step is not necessary )
target_twiss_at_ENDHTR = tao.ele_twiss('ENDHTR')  # Record the undistorted twiss downstream 

target_twiss_at_ENDHTR

dt: 0.11398601531982422


In [ ]:
# Distort the QUAD (Q0H01) on the real machine, and update bmad live-model again
# The downstream optics in live-model should also be distorted

run1(tao, MONITOR, DM_GOOD)

In [93]:
dat_name = "HTR.endtwiss"
var_name = "q.HTR[5:9]"
desired_twiss = target_twiss_at_ENDHTR

# this function runs the optimizer!!!
dict_new_quads = twiss_optimizer2(dat_name, var_name, desired_twiss, reset_lat=False)
dict_new_quads

Old merit: 3.66047060055099e-28
New merit: 8.61436108501345e-26


{'Q0H05': 1.99047760958474,
 'Q0H06': -0.489082889935051,
 'Q0H07': -2.08005818801471,
 'Q0H08': 2.20748978167364,
 'QHD01': 2.33337098172311}

### Assuming Tao optimizer was able to recover the optics at ENDHTR,
### we need to upload the quad strengths (the five quads) to the actual machine via `caput_many`.
### Datamap is required again for conversion.

In [83]:
def prepare_caput_values(dict_quads, df):
    """
    Given a dictionary of quad names (Bmad names) and values,
    Provide a dictionary of the corresponding quad PV names and PV values. 
    """
    
    quad_names = dict_quads.keys()
    quad_values = dict_quads.values()
    
    df2 = df[df['bmad_name'].isin(quad_names)][['bmad_name','pvname','bmad_factor']]
    print(df2)
    
    out = {}
    
    # temporary look-up dictionary (to make sure order does not matter)
    q_value = {}
    for name, value in zip(quad_names, quad_values):
        q_value[name] = value
    
    for row in df2.itertuples():
        
        key = row.pvname
        value = 1.0/row.bmad_factor * q_value[row.bmad_name]
        
        assert key not in out  # Check no repeated keys (quad PVs)
        out[key] = value
    return out

### Dictionary to caput_many is here!

In [41]:
df = DM_GOOD['quad'].data;

dict_to_caput = prepare_caput_values(dict_new_quads, df)
dict_to_caput

  bmad_name             pvname  bmad_factor
5     Q0H05  QUAD:HTR:365:BDES    -0.803859
6     Q0H06  QUAD:HTR:385:BDES    -0.803859
7     Q0H07  QUAD:HTR:440:BDES    -0.803859
8     Q0H08  QUAD:HTR:460:BDES    -0.803859
9     QHD01  QUAD:HTR:830:BDES    -0.803859


{'QUAD:HTR:365:BDES': -6.829562287617346,
 'QUAD:HTR:385:BDES': -0.022527611493915807,
 'QUAD:HTR:440:BDES': 3.884894913212843,
 'QUAD:HTR:460:BDES': -3.352605432177624,
 'QUAD:HTR:830:BDES': -2.7584432523164786}

### Revert to the old quad values ( in case the new values fail on the real machine )
### These cells need to be run BEFORE you upload the new values

In [53]:
original_set_values = caget_many(dict_to_caput.keys())
original_set_values

[-2.4766963, 0.610201, 2.5876414, -2.7461416, -2.9026848]

In [54]:
# Caput this dictionary to recover old setting
old_dict_to_caput = {list(dict_to_caput.keys())[i]:original_set_values[i] for i in range(len(original_set_values))}
old_dict_to_caput

{'QUAD:HTR:365:BDES': -2.4766963,
 'QUAD:HTR:385:BDES': 0.610201,
 'QUAD:HTR:440:BDES': 2.5876414,
 'QUAD:HTR:460:BDES': -2.7461416,
 'QUAD:HTR:830:BDES': -2.9026848}